In [1]:
pip install spotipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 2.2 MB/s eta 0:00:00


In [2]:
import sys
import spotipy
import yaml
import spotipy.util as util
from pprint import pprint
import json
import argparse
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import seaborn as sns
from yellowbrick.cluster import KElbowVisualizer
#from kneed import KneeLocator
import plotly.graph_objects as go
from plotly.subplots import make_subplots
sns.set()

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up the credentials
client_id = 'ad21dd08176345fc8375e20bbf37724a'
client_secret = 'a43bd4b2c3f64b4fa677f64705e82a7d'

# Authenticate
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Use the API
results = sp.search(q='artist:Daft Punk', type='artist')
print(results)

{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3ADaft+Punk&type=artist&offset=0&limit=10', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4tZwfgrHOc3mvqYlEYSvVi'}, 'followers': {'href': None, 'total': 9920674}, 'genres': ['electro', 'filter house', 'rock'], 'href': 'https://api.spotify.com/v1/artists/4tZwfgrHOc3mvqYlEYSvVi', 'id': '4tZwfgrHOc3mvqYlEYSvVi', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5eba7bfd7835b5c1eee0c95fa6e', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174a7bfd7835b5c1eee0c95fa6e', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178a7bfd7835b5c1eee0c95fa6e', 'width': 160}], 'name': 'Daft Punk', 'popularity': 79, 'type': 'artist', 'uri': 'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/3koSTh1rFReMYVMCwZTLZG'}, 'followers': {'href': None, 'total': 296}, 'genres': [], 'href': 'ht

In [4]:
def get_playlist_info(username, playlist_uri):
    # initialize vars
    offset = 0
    tracks, uris, names, artists = [], [], [], []

    # get playlist id and name from URI
    playlist_id = playlist_uri.split(':')[2]
    playlist_name = sp.user_playlist(username, playlist_id)['name']

    # get all tracks in given playlist (max limit is 100 at a time --> use offset)
    while True:
        results = sp.user_playlist_tracks(username, playlist_id, offset=offset)
        tracks += results['items']
        if results['next'] is not None:
            offset += 100
        else:
            break

    # get track metadata
    for track in tracks:
        names.append(track['track']['name'])
        artists.append(track['track']['artists'][0]['name'])
        uris.append(track['track']['uri'])

    return playlist_name, names, artists, uris

In [5]:
# Extract features from each track in a playlist
def get_features_for_playlist(df, username, uri):

    # get all track metadata from given playlist
    playlist_name, names, artists, uris = get_playlist_info(username, uri)

    # iterate through each track to get audio features and save data into dataframe
    for name, artist, track_uri in zip(names, artists, uris):
        # print(json.dumps(track_uri, indent=4))
        # ^ DEBUG STATEMENT ^

        # access audio features for given track URI via spotipy
        audio_features = sp.audio_features(track_uri)

        # get relevant audio features
        feature_subset = [audio_features[0][col] for col in df.columns if col not in ["name", "artist", "track_URI", "playlist"]]

        # compose a row of the dataframe by flattening the list of audio features
        row = [name, artist, track_uri, *feature_subset, playlist_name]
        df.loc[len(df.index)] = row
    return df

In [6]:
df = pd.DataFrame(columns=['name', 'artist', 'track_URI', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'playlist'])


In [8]:
# Assuming user_config is a dictionary with necessary user information
user_config = {
    'username': 'Srithijaa Sankepally'
}

# Assuming the function get_features_for_playlist is defined elsewhere in your code
df = get_features_for_playlist(df, user_config['username'], 'spotify:playlist:3VZCYCHOtahASqlkxE8vJx')
df


,name,artist,track_URI,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,playlist
0,Way With Words,Bahamas,spotify:track:6ioeN8jQ8Qc3tS8dhpoAVi,0.35900,0.785,0.530,0.001980,0.0774,-6.995,0.0634,146.380,0.7290,twenty
1,Caroline,Briston Maroney,spotify:track:3S5mohVxC0Xuj0tgZ7vU7g,0.72600,0.506,0.456,0.000037,0.1100,-10.635,0.0479,149.600,0.4060,twenty
2,Hear Me Calling,Juice WRLD,spotify:track:13ZyrkCDmRz5xY3seuAWYk,0.30800,0.699,0.687,0.000036,0.1210,-3.997,0.1060,88.932,0.4990,twenty
3,94 Bentley,SAINt JHN,spotify:track:0xxbDtdmc88FOV7YvpEtft,0.07920,0.696,0.558,0.000000,0.1620,-5.253,0.0512,113.000,0.2170,twenty
4,I Feel Like I'm Drowning,Two Feet,spotify:track:5h0ODSutX73Vt4vZsmmph0,0.70300,0.719,0.575,0.003340,0.0899,-5.436,0.0460,104.892,0.2340,twenty
...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,Night Mode,UnoTheActivist,spotify:track:3pi1QC6F2BwSagRqejpXnt,0.00676,0.670,0.715,0.050700,0.6530,-10.001,0.1670,132.056,0.6750,twenty
488,Tyler Herro,Jack Harlow,spotify:track:4DuUwzP4ALMqpquHU0ltAB,0.11000,0.794,0.756,0.000000,0.2470,-7.160,0.1360,123.066,0.7750,twenty
489,BLACK PARADE,Beyoncé,spotify:track:2qzUpSVI4NnPyWxbXwumTj,0.00337,0.824,0.555,0.000006,0.2170,-5.817,0.0500,120.002,0.0611,twenty
490,Spark,Forester,spotify:track:22wa6vtAb3FVnBm3dUjmQH,0.59200,0.339,0.405,0.000000,0.1250,-9.590,0.1920,179.751,0.0504,twenty
